# Task 2

In this notebook, the following has been done:
1. Get all images from a folder
2. For each image, computation of Color moments.
3. For each image, computation of Extended Local binary patterns
4. For each image, computation of Histograms of oriented gradients

In [49]:
# Imports

import os
import csv
import scipy
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from skimage.feature import hog, local_binary_pattern

In [50]:
# Global Constants

CURRENT_PATH = Path(os.getcwd())
IMGS_DIR = CURRENT_PATH.parent / "Outputs" / "Task0" / "Images"

WRITE_DIR =  CURRENT_PATH.parent / "Outputs" / "Task2" 

#### Cell contains helper functions for Color Moments

In [54]:
def create_windows(np_image):
    # Creating windows over the image
    windows = []
    for x in range(0,64,8):
        for y in range(0,64,8):
            window = np_image[x:x+8, y:y+8]
            windows.append(window)    
    return np.array(windows)

def calculate_color_moments(windows=None):
    
    # Exceptions
    if (windows is None):
        raise Exception('No image passed as parameter')
    
    # Calculating color moments over windows
    means = []
    stds = []
    skews = []
    for index in range(0,len(windows)):
        # Extracting window
        window = windows[index]
        flatten_window = window.flatten()
    
        # Calculating mean, std deviation and skewness for window
        mean = np.mean(flatten_window)
        std = np.std(flatten_window)
        skew = scipy.stats.skew(flatten_window)
    
        # Appending results of each window to a list
        means.append(mean)
        stds.append(std)
        skews.append(skew)
        
    return means, stds, skews

def write_color_moments(means, stds, skews, windows, path=None):
    if path is None:
        raise Exception("No write path for color moments")
    
    with open(path, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(["WindowID", "Mean", "Std. Deviation", "Skewness"])
    
        for window_id, mean, std, skew in zip(range(0, len(windows)), means, stds, skews):
            csvwriter.writerow([window_id, mean, std, skew])

#### Cell contains helper functions for ELBP

In [55]:
def calculate_elbp(np_image, points=None, radius=None, method=None):
    if points is None or radius is None or method is None:
        raise Exception("No points / radius / method")
        
    elbp = local_binary_pattern(np_image, P=points, R=radius, method=method)

    # Bining the results of ELBP
    bins = 2**points
    bining_results = np.histogram(elbp, bins=bins, range=(0.0, float(bins)))
    
    return bins, bining_results

def write_elbp(bins, bining_results, path=None):
    if path is None:
        raise Exception("No write path for color moments")
        
    # Writing results into a file
    with open(path, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(["ELBP Value", "Frequency"])

        for elbp_value, frequency in zip(range(0, bins), bining_results[0]):
            csvwriter.writerow([elbp_value, frequency])

#### Cell contains helper functions for HOG

In [60]:
def calculate_hog(np_image, orientations, pixels_per_cell, cells_per_block):
    if (np_image is None or orientations is None or pixels_per_cell is None or cells_per_block is None):
        raise Exception("No orientations / pixels_per_cell / cells_per_block")
        
    fd, hog_image = hog(np_image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)
    return fd, hog_image

def write_hog(path, feature_values):
    with open(path, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(["HOG Feature Vector"])
    
        for hog_value in feature_values:
            csvwriter.writerow([hog_value])

### Main Program

In [61]:
img_file_names = [f for f in os.listdir(IMGS_DIR) if os.path.isfile(os.path.join(IMGS_DIR, f))]
img_file_names.sort()

for file_name in img_file_names:
    
    # Preparing all paths related to the file
    file_base_name = os.path.splitext(file_name)[0]
    file_path = IMGS_DIR / img_file_name
    write_dir = WRITE_DIR / file_base_name
    write_dir.mkdir(parents=True, exist_ok=True)
    write_color_moments_path = WRITE_DIR / file_base_name / "color_moments.csv"
    write_elbp_path = WRITE_DIR / file_base_name / "elbp.csv"
    write_hog_path = WRITE_DIR / file_base_name / "hog.csv"
    
    # Extracting the image in an nparray for analysis
    img = Image.open(str(image_path))
    np_img = np.array(image)
    
    # Calculating Color Moments of an Image
    windows = create_windows(np_image)
    means, stds, skews = calculate_color_moments(windows)
    write_color_moments(means, stds, skews, windows, write_color_moments_path)

    # Calculating ELBP of an Image
    features, _ = calculate_hog(np_image, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2))
    write_hog(write_hog_path, features)